# 6장 지지 벡터 머신_p128~p163

## < 학습 목표 >
* SVM 소개
 - SVM의 특징 및 주요 개념 소개
* 최적화를 위한 SMO 알고리즘
 - 가장 인기가 좋은 SVM 방법 중 하나인 SMO(Sequential Minimal Optimization) 알고리즘 소개
* 데이터 '변형'을 위해 커널 사용하기
 - 선형 분리가 불가능한 데이터 집합을 분류하기 위한 kernel trick 소개
* 다른 분류와 지지 벡터 머신 비교하기
 - 필기체 인식 예제에서의 kNN과 SVM 성능 비교

##  <span style="color:#1589FF"><b>SVM의 특징</b></span>
### ■ SVM의 차별성
* 신경망을 포함한 기존 방법들은 분류 '오류율을 최소화'하려는 목적으로 설계
* 하지만, SVM은 <b>두 부류 사이에 존재하는 '여백을<sup>margin</sup> 최대화'하여 일반화 능력을 극대화함</b>
<br>(※ 일반화 능력 : 미지의 패턴을 얼마나 잘 분류하느냐를 따지는 척도)

### ■장점
* 일반화의 오류가 낮음
* 결과 해석이 쉬움

### ■단점
* 분류 결과가 매개변수 조정과 kernel 선택에 민감함 
<br>(※ kernel : 선형 분류가 불가능한 경우에 데이터를 더 높은 차원으로 확장시키기 위한 일종의 함수)

### ■SVM은 구현이 까다로운 편
* SVM은 N<sup>2</sup>개 이상의 항을 갖는 수식을 다루어야 함
* 주어진 조건 하에서 수식을 최대화하는 조건부 최적화 문제를 풀어야 함

### ■활용
* 수치형
* 명목형

##  <span style="color:#1589FF"><b>SVM의 발상</b></span>
* 세 개의 분류기 (결정 직선)를 보여주고 있음
* 기존 분류 방법(ex. 신경망 학습 알고리즘)은 초기값으로 ①을 가지고 출발하였다면 오류를 줄여 가는 과정을 반복하여 ② 또는 ③과 같은 직선을 찾아 감
* 만일 ②에 도달하였다면 거기에서 멈춤 
 - (why?)> 오류 함수가 0의 값을 갖게 되므로 추가로 ③과 같은 분류기를 찾아갈 이유가 없기 때문
 - 훈련 입장에서 보면 ②와 ③은 둘 다 오류가 0이므로 같은 성능의 분류기
* 하지만, 분류기를 만드는 목적은 미래에 발생할 미지의 패턴을 분류하기 위함에 있음
 - 즉, 미지의 패턴을 얼마나 잘 분류하느냐를 따지는 척도인 일반화<sup>generalization</sup> 능력 측면에서 보면②와 ③은 갖지 않음
 - ②는 발생하는 패턴에서 약간의 변형이 발생하면 결정 직선을 넘어 오분류될 가능성이 큼
 - 하지만, 직선 ③은 두 부류 모두에 대해 여백<sup>margin</sup>이 커서 웬만한 변형은 오분류로 이어지지 않음
 - 따라서, ②에 비해 ③이 일반화 능력이 뛰어나고 결국 분류기 품질 면에서 더 좋다고 할 수 있음~!! 
<img src="../src/Ch06_SVM/Img/classifier.jpg">

##  <span style="color:#1589FF"><b>선형 SVM</b></span>

### ■ <b> d(x)=w<sup>T</sup>x+b=0 </b>  
 - 이진 분류를 위한 결정 초평면<sup> decision hyperplane </sup> 표현
 - x: 샘플을 나타내는 특징 벡터
 - w, b: 결정 초평면을 정의하는 매개 변수
 <br> (※ <span style="color:red"><b>SVM 학습이란?</b></span> w와 b를 구하는 과정이고, 라그랑제 승수 α를 구하면 됨)
 
#### 초평면의 SVM을 유도하는데 도움이 되는 몇 가지 수학적 특성
 1. d(x)는 전체 특징 공간을 두 영역으로 분할하며, 한 쪽 영역에 속하는 점 x는 d(x)>0이고 다른 쪽에 있는 점은 d(x)<0 이다
 2. 하나의 초평면을 표현하는 식은 여럿 있다. 위 식에 0이 아닌 임의의 상수 c를 곱하여도 같은 초평면을 나타낸다
 3. w는 초평면의 법선 벡터로서 <sup>normal vector</sup> 초평면의 방향을 나타내고, b는 위치를 나타낸다
 4. 임의의 점 x에서 초평면까지의 거리는 다음과 같음
<img src="../src/Ch06_SVM/Img/2.jpg">

###  <span style="color:#6AFB00"><b>1. 선형 분리 가능한 상황</b></span>

* w(결정 직선의 방향)가 주어진 상황에서,
 - b를 변화시키면 그에 따라 직선의 위치가 변함
 - '두 부류에 대해 직선으로부터 가장 가까운 샘플까지의 거리가 같게 되는' b를 결정
 <br> (그림에서 두 개의 결정 직선 ①과 ②는 이렇게 얻은 것)
 - <span style="color:Blue"><b>여백(마진)<sup>margin</sup></b></span>은 직선으로부터 가장 가까운 샘플까지 거리의 두 배로 정의함
 - 가장 가까운 샘플을 <span style="color:Blue"><b>서포트 벡터</b></span>라 부름
<img src="../src/Ch06_SVM/Img/4.jpg">
<br><br>
* <b> 여백을 가장 크게 하는 결정 초평면의 방향, 즉 w를 찾는 것이 중요함~!! </b>
 - 최적을 찾기 위해서는 수학에 기대는 수밖에 없음~!!
* 속이 찬 샘플은 직선으로부터 가장 가까운 것들로서 여백의 크기를 좌우하는 중요한 역할을 함 (<span style="color:Blue"><b>서포트 벡터<sup>support vector</sup></b></span>)
* 여백의 공식화
<img src="../src/Ch06_SVM/Img/5.jpg">
* <b>조건부 최적화 문제<sup>constrained optimization problem</sup></b>
 <br> (모든 샘플을 옳게 분류한다는 조건 하에 최대 여백을 갖는 결정 초평면 찾는 문제)
 - 문제를 조건부 최적화 문제로 공식화
 <br> (조건 식에서 등호가 성립하는 샘플이 서포트 벡터)
<img src="../src/Ch06_SVM/Img/6.jpg">
<br><br>
* 위 식의 조건부 최적화 문제를 간단한 형태로 변형하고, 최소화 문제로 바꾸어 쓸 수 있음
 - 1/||w||의 최대화는 ||w||<sup>2</sup>의 최소화와 같음
<img src="../src/Ch06_SVM/Img/7.jpg">

<br>
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++<br>
<span style="color:Red"><b> 최대 여백을 확보함으로써 일반화 능력을 극대화하겠다는 SVM의 철학을 수학식으로 표현한 것</b></span>
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

#### 문제의 특성
* 해의 유일성<sup>uniqueness</sup>
 - N개의 조건식이 모두 선형이고 이들 모두를 만족하는 영역은 볼록이 되기 때문에 해는 유일하다
 - 구한 해는 전역 최적점을 보장한다
* 문제의 난이도
 - N개의 선형 부등식을 조건으로 2차 함수의 최적화 문제

#### 수학에서 이런 종류의 조건부 최적화 문제는 <span style="color:Blue"><b>라그랑제 승수<sup>Lagrange multiplier</sup></span>를 도입하여 해결</b>
* <span style="color:Blue"><b>라그랑제 승수<sup>Lagrange multiplier</sup></b></span>
 - 제약이 있는 최적화 문제에서 원 문제(primal problem)를 풀기 쉬운 문제(dual problem)로 변환하는 기법 
 - 어떤 함수가 주어진 제약식을 만족시키면서 그 함수가 갖는 최대값 혹은 최소값을 찾고자 할 때 사용
 - <b> 핵심) 두 가지 조건을 동시에 만족시키고 공통접선을 찾는 것 </b>

#### 라그랑제 함수로 표현한 조건부 최적화 문제는 보통 <span style="color:Blue"><b>Karush-Kuhn-Tucker(KKT) 조건</b></span>을 이용하여 품
* KKT 조건
<img src="../src/Ch06_SVM/Img/8.jpg">
 - 4번째 식에 의하면, 모든 샘플이 α<sub>i</sub>=0 또는 t<sub>i</sub>(w<sup>T</sup>x<sub>i</sub>+b)-1=0 이어야 함
 <br> α<sub>i</sub>≠0인 샘플이 <span style="color:Blue"><b>서포트 벡터</b></span>임
 - 1번째 식에 의하면, 라그랑제 승수 α<sub>i</sub> 알면 w 구할 수 있음 
 <br> (이제부터 'w를 구하는 대신 라그랑제 승수를 구하는' 문제로 관심 전환)
 - 4번째 식에 의하여 b를 구할 수 있음

#### 문제의 볼록 성질을 이용하여 풀기 쉬운 형태로 변환
* 볼록 성질을 만족하는 조건부 최적화 문제는 <span style="color:Blue"><b>Wolfe 듀얼<sup>dual</sup></b></span> 문제로 변형할 수 있음
<img src="../src/Ch06_SVM/Img/10.jpg">
<br>
* 위의 조건부 최적화 문제를 Wolfe 듀얼로 바꾸어 쓰면,
<img src="../src/Ch06_SVM/Img/11.jpg">
 - 부등식 조건이 등식 조건으로 바뀌어 문제가 훨씬 풀기 쉽기 때문에 문제 변형으로 얻은 것이 크다고 할 수 있음

<br>
++++++++
#### <span style="color:Red"><b>최종적으로,</b></span>  간단한 수식 정리를 하면
++++++++
<img src="../src/Ch06_SVM/Img/12.jpg">
* 흥미로운 특성들
 - 2차 함수의 최대화 문제
 - w와 b가 사라짐 (α를 찾는 문제가 되었음)
 - 특징 벡터 x<sub>i</sub>가 내적 형태로 나타남 (비선형으로 확장하는 발판)
 - 목적 함수의 두번째 Σ항은 N<sup>2</sup>개의 항을 갖음 (여전히 풀기 어려운 문제)

### <span style="color:Red"><b>■ 조건부 최적화 문제의 변형 과정</b></span>
<img src="../src/Ch06_SVM/Img/9.jpg">

#### ■ [예제 5.2] 두 개의 샘플을 갖는 경우에 대한 문제의 풀이
<img src="../src/Ch06_SVM/Img/ex.5.2.jpg">

#### ■ [예제 5.3] 세 개의 샘플을 갖는 경우에 대한 문제의 풀이
<img src="../src/Ch06_SVM/Img/ex.5.3.jpg">

###  <span style="color:#6AFB00"><b>2. 선형 분리 불가능한 상황</b></span>

* 샘플 (x,t)의 세 가지 상황
<img src="../src/Ch06_SVM/Img/5.2.2_fig.5.6.jpg">

* 슬랙 변수 ξ를 도입하여 하나의 식으로 쓰면,
<img src="../src/Ch06_SVM/Img/5.2.2_table.5.2.jpg">

<img src="../src/Ch06_SVM/Img/5.2.2_fig.5.7.jpg">
<img src="../src/Ch06_SVM/Img/5.2.2_fig.5.8.jpg">

<img src="../src/Ch06_SVM/Img/5.2.2_fig.5.9.jpg">

<img src="../src/Ch06_SVM/Img/5.2.2_fig.5.10.jpg">

##  <span style="color:#1589FF"><b>비선형 SVM</b></span>

### ■ 비선형 SVM으로 확장
* 언뜻 어려운 작업처럼 보이지만, 의외로 굉장히 간단함
* 커널(kernel)을 활용하여 간단히 풂
* 커널을 적용할 수 있는 이유는 식(5.27)에서 <b>'특징 벡터가 내적 형태'</b>로만 나타나기 때문임


###  <span style="color:#6AFB00"><b>1. 커널 대치 (Kernel Substitiution)</b></span>

* 특징 공간에서는 선형분리가 가능하지 않은데, 이 공간을 더 높은 차원의 새로운 공간으로 매핑하여 <b>선형 분리 가능</b>하게 만들 수 있음

<img src="../src/Ch06_SVM/Img/5.3.1_example.5.5.jpg">

<img src="../src/Ch06_SVM/Img/5.3.1_fig.5.8.jpg">

### ■ <b> 공간매핑 </b>  
 - 공간 매핑은 아래 식과 같이 표기할 수 있음
 - <b>공간 L</b>은 원래 특징 공간이고, <b>공간 H</b>는 매핑된 새로운 공간임

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.28.jpg">

- SVM이 사용하는 커널함수의 성질

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.29.jpg">

<img src="../src/Ch06_SVM/Img/5.3.1_example.5.6.jpg">

### ■ 커널 대치 (kernel substitution)
* 다른 말로 커널 트릭 (kernel trick)이라고도 함
* 어떤 수식이 벡터 내적을 포함할 때, 그 내적을 <b>커널 함수</b>로 대치하여 계산하는 기법
 - 실제 계산은 공간 L에서 K()의 계산으로 이루어짐
 - 고차원 공간 H에서 작업하는 효과를 가짐
 - (적용 예) Fisher LD의 kernel LD로의 확장, PCA를 kernel PCA로 확장

### ■SVM에 적용 가능
- 식(5.13) 과 식(5.27)에서 벡터의 내적 형태로 나타나기 떄문에 적용 가능함
- 실제 계산은 공간 L에서 이루어지지만 분류는 선형 분류에 유리한 H에서 수행함
- 이러한 기법을 이용하여 <b>차원의 저주</b>를 피함



###  <span style="color:#6AFB00"><b>2. 커널 대치를 이용한 비선형 SVM</b></span>

### ■커널 대치를 이용한 비선형 SVM
* 커널 대치를 이용하면, 선형 SVM을 아주 쉽게 비선형 SVM으로 확장할 수 있음
* 공간 L에서 작업하는 대신, <b>매핑함수</b>를 통하여 공간 H로 매핑한 후 해당 공간에서 작업을 진행하면 됨
* (예) 그림 5.8에서 공간 L의 샘플 a, b, c, d를 분류하는 대신, 공간 H에서 a,b,c,d를 분류하는 셈임

<img src="../src/Ch06_SVM/Img/5.3.1_fig.5.8.jpg">

* 이 방법을 통하여, 5.2절의 최적화 문제 (5.13)과 (5.27)에 있는 목적함수를 다음 식(5.30)처럼 바꾸어 쓸 수 있음

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.30.jpg">

* 또한, 커널 함수의 성질을 이용하여 식(5.30)을 다음 식(5.31)로 바꾸어 쓸 수 있음

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.31.jpg">

### ■어떤 커널 함수를 사용할 것인가?
* 커널 대치 관련하여 남은 마지막 하나
* SVM이 사용하는 대표적인 커널 함수는 아래 (5.32)~(5.34)와 같음

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.32.jpg">

### ■짚고 넘어가야 할 점
* 어떤 커널함수의 경우
 - 커널 함수 K에 대해서 조건을 만족하는 매핑함수가 존재한다는 사실은 증명되어 있음
 - 그러나 매핑 함수가 어떻게 생겼는지도 모름
 - 심지어 공간 H의 차원이 얼마인지도 모름
 
 
 
* 그럼에도 불구하고, 해당 커널 함수 K는 사용가능함
 - 그 이유는 지금까지 유도한 식을 보면 알 수 있음
 - SVM을 만들기 위해 풀어야하는 식은 (5.30)이며, 이 식에 벡터 내적만 나타나있으므로 (5.31)로 바꾸어 계산하면 됨
 - 또한 인식을 하는 단계에서도 벡터의 내적만 나타남



* 즉, 어떤 커널 함수에 대응하는 매핑 함수 자체를 모르더라도 <b>'존재한다는 사실'</b>만 알면, 그 커널을 사용할 수 있음

##  <span style="color:#1589FF"><b>구현</b></span>

###  <span style="color:#6AFB00"><b>1. 학 습</b></span>

### ■ 학습이란
* 학습이란, (5.1)에 있는 w와 b를 구하는 과정임
 - 이들 값은 조건부 최적화 문제 (5.27)을 풀어서 구해야 함 (선형 SVM)
 - 그러나 선형 SVM으로 만족할만한 성능이 보장되지 않는다면, <b>비선형 SVM 문제</b>를 풀어야함
 
 
* 비선형 SVM 학습을 위한 조건부 최적화 문제를 (5.31)이용하여 다시쓰면, 다음 (5.35)과 같음

<img src="../src/Ch06_SVM/Img/5.3.1_form.5.35.jpg">

* 이 조건부 최적화 문제를 풀어, <b>라그랑제 승수</b> (a, 1 < i < N)를 모두 찾아내면 학습이 끝남

### ■ 비선형 SVM 문제 풀이의 어려움
* 목적함수의 첫째항은 N개의 1차항, N^2개의 2차항을 가진 매우 복잡한 수식임
* (예) CENPARMI 필기 숫자 DB의 경우 N = 4000임
 - 즉, 4000개의 1차항과 8002000개의 2차 항을 가지 매우 긴 식임
 - 또한 4000개의 부등식 조건이 있고, 4000개의 1차항을 가진 한개의 등식 조건이 있음
 
 
* 결론적으로 문제를 풀기에 굉장히 까다로움
 - N이 작은 경우에만 <b>분석적</b>으로 풀 수 있고, 이외에 경우에는 <b>수치적</b>으로 풀어야 함

### ■ 문제를 푸는 기본전략
* 다룰 수 있을 정도의 작은 문제로 분해하는 것
 - N개의 라그랑제 승수를 <b>활성화 집합 Y</b>와 <b>비활성화 집합 Z</b>로 나눔
 - Z에 속한 라그랑제 승수는 고정된 상수 값으로 간주하고, <b>Y에 대해서 최적화</b>를 수행함
 
 
* 대략적인 알고리즘은 다음과 같이 기술할 수 있음

<img src="../src/Ch06_SVM/Img/5.3.1_alg.5.1.jpg">

* 이 방법을 구현하기 위해서는 세가지를 결정해야 함
 - 첫째, <b>q를 얼마로 할 것인가</b>와 <b>Y를 어떻게 선택할 지</b> 결정해야 함
 - 둘째, <b>분해된 문제를 푸는 방법</b>을 고안해야 함
 - 셋째, 최적화 조건이 만족되었는지 여부를 <b>판단하는 조건</b>이 있어야 함

### ■ SMO (Sequential Minimal Optimization) 알고리즘
* 위 기본전략에 따른 풀이 알고리즘 중 가장 널리 알려진 알고리즘 임
* q가 가질 수 있는 가장 작은 값(2)를 사용함
 - 따라서, 두개의 라그랑제 승수만을 변수로 갖는 문제로 축소되며, 이 문제는 <b>분석적</b>으로 풀 수 있음
* 알고리즘의 종료 조건으로 <b>KKT 조건</b>을 사용함 

## 6.3 SMO 알고리즘으로 효율적인 최적화하기

* 이전에 사람들은 최적화 문제를 해결하기 위해서 이차 방정식 솔버를 사용


* 이차 방정식 솔버(quadratic solver)
 - 선형적인 제약 조건이 있는 여러 변수를 가지고 이차 방정식 함수를 최적화하는 소프트웨어의 한 부분
 - 강력한 계산을 가지며 복잡함
 - 최적의 a값을 찾으므로써 분리 초평면 또는 2D에서 선을 구할 수 있으며, 데이터를 쉽게 분류할 수 있게 됨

먼저 SMO알고리즘에 대해 논의한 후, 이를 <b>간단한 형태</b>로 표현
이후, <b>완전한 형태</b>의 알고리즘을 구현하여 이해를 도움

### 6.3.1 플랫의 SMO 알고리즘


* 순차적 최소 최적화
* 커다란 최적화 문제를 가짐
 * 이 문제는 다시 작은 문제들로 나누어지며, 이들은 쉽게 해결 할 수 있으며 순차적으로 해결 가능
 * 순차적으로 해결된 문제들의 답은 모두 같게 됨
 * 이는 모든 문제를 다함께 처리하는 것과 같은 효과가 있음
 * 결론적으로, 같은 답을 얻게 됨으로써 처리 시간이 크게 줄어듦
 

* SMO 알고리즘은 a와 b의 집합을 찾는 것
 - 알파의 집합을 구하게 되면 가중치 w는 쉽게 계산되며 분리 초평면을 구할 수 있음


* <b> SMO 알고리즘의 동작 방식 </b>
 1. 각각의 사이클을 최적화하기 위하여 두 개의 알파를 선택
 2. 적당한 알파의 쌍을 찾게 되면, 그 중 하나는 값을 증가시키고 다른 하나는 줄임
  <br> (※ 적당한 알파를 구하기 위해서는 알파 집합이 조건을 정확하게 충족해야 함)
 3. 첫 번째 조건은 알파 쌍 모두가 그들의 마진 경계 밖에 있어야 한다는 것
 4. 두 번째 조건은 알파가 이미 고정되어 있거나 경계를 갖지 않아야 한다는 것

### 6.3.2 간략한 형태의 SMO로 적은 양의 데이터 집합 해결하기

* 간략화 알고리즘은 적은 양의 코드를 사용하지만 실행 시간이 오래 걸림
* 플랫 SMO 알고리즘의 바깥쪽 반복문에서는 최적화를 위해 가장 좋은 알파를 결정
* 간략화 형태의 알고리즘에서는 이 부분을 건너뛰고 데이터 집합에 있는 모든 알파를 대상으로 알파의 쌍을 먼저 선택
* 그런 다음, 남아 있는 알파들에서 임의로 두 번째 알파 선택
<br> (※ 여기서 주의해야 할 점은 두 알파를 동시에 변경해야 한다는 것)
 - 다음과 같은 제약 조건이 있기 때문
<img src="../src/Ch06_SVM/Img/제약조건.JPG">
* 알파 하나만을 변경하는 것은 제약 조건을 위반하는 것이므로 <b>항상 알파 두 개를 한꺼번에 변경</b>
 - 이를 수행하기 위해 <b>도움 함수</b>를 생성
 - 이 함수는 범위 내에서 임의로 하나의 정수를 선택
 - 또한, 너무 큰 값을 얻지 않도록 값을 고정하는 도움 함수도 필요

In [26]:
%cd c:/Users/BMW/Documents/

c:\Users\BMW\Documents


In [27]:
import os
srcDir = "C:\Users\BMW\machinelearninginaction-master"
srcDir2wd = os.path.join(srcDir,'Ch06')

In [28]:
%cd {srcDir2wd}

C:\Users\BMW\machinelearninginaction-master\Ch06


#### 리스팅 6.1 SMO 알고리즘을 위한 도움 함수

In [1]:
from numpy import *
from time import sleep

# 도움 함수 #1
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

# 도움 함수 #2
"""
i는 첫 번째 알파의 색인; m은 알파 전체의 개수
임의로 선택된 하나의 값은 i와 동일하지 않을 때까지 반복하여 선택한 값으로 반환
"""
def selectJrand(i,m): 
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

# 도움 함수 #3
"""
알파 값이 H보다 크거나 L보다 작은 값일 경우 더 크거나 더 작아지지 않도록 고정시킴

"""
def clipAlpha(aj,H,L): # 도움 함수 #2
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

In [37]:
dataArr, labelArr = loadDataSet('testSet.txt')

# labelArr
print labelArr

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]


 <span style="color:Red">※ 분류 항목 표시가 0과 1이 아닌 -1과 1인 것을 확인할 수 있는데,
  <br>  -1과 1은 단지 부호만 다르기 때문에 계산을 관리할 수 있음 </span> 

In [38]:
# dataArr
print dataArr

[[3.542485, 1.977398], [3.018896, 2.556416], [7.55151, -1.58003], [2.114999, -0.004466], [8.127113, 1.274372], [7.108772, -0.986906], [8.610639, 2.046708], [2.326297, 0.265213], [3.634009, 1.730537], [0.341367, -0.894998], [3.125951, 0.293251], [2.123252, -0.783563], [0.887835, -2.797792], [7.139979, -2.329896], [1.696414, -1.212496], [8.117032, 0.623493], [8.497162, -0.266649], [4.658191, 3.507396], [8.197181, 1.545132], [1.208047, 0.2131], [1.928486, -0.32187], [2.175808, -0.014527], [7.886608, 0.461755], [3.223038, -0.552392], [3.628502, 2.190585], [7.40786, -0.121961], [7.286357, 0.251077], [2.301095, -0.533988], [-0.232542, -0.54769], [3.457096, -0.082216], [3.023938, -0.057392], [8.015003, 0.885325], [8.991748, 0.923154], [7.916831, -1.781735], [7.616862, -0.217958], [2.450939, 0.744967], [7.270337, -2.507834], [1.749721, -0.961902], [1.803111, -0.176349], [8.804461, 3.044301], [1.231257, -0.568573], [2.074915, 1.41055], [-0.743036, -1.736103], [3.536555, 3.96496], [8.410143, 0.0

#### 리스팅 6.2 간략한 형태의 SMO 알고리즘

In [2]:
# 이 함수는 5개 인자 (데이터 집합, 분류 항목 표시, 상수 C, 오차 범위, 종료하기 전 최대 반복 횟수) 처리
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn); labelMat = mat(classLabels).transpose()
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0
    # 1. 알파가 변경될 수 있다면 최적화 입력
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])#if checks if an example violates KKT conditions
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                # 2. 임의로 두 번째 알파 선택
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                # 3. 알파가 0과 C 사이의 값이 되도록 함
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print "L==H"; continue
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0: print "eta>=0"; continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                # 4. 반대쪽에 있는 j와 동일하게 계산된 값으로 i를 갱신
                if (abs(alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                        #the update is in the oppostie direction
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                # 5. 상수의 간격을 설정
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print "iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print "iteration number: %d" % iter
    return b,alphas

In [50]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

L==H
L==H
iter: 0 i:2, pairs changed 1
iter: 0 i:3, pairs changed 2
j not moving enough
L==H
L==H
L==H
L==H
L==H
L==H
L==H
j not moving enough
iter: 0 i:18, pairs changed 3
L==H
iter: 0 i:26, pairs changed 4
j not moving enough
iter: 0 i:29, pairs changed 5
j not moving enough
L==H
j not moving enough
j not moving enough
iter: 0 i:53, pairs changed 6
j not moving enough
L==H
iter: 0 i:69, pairs changed 7
iteration number: 0
iter: 0 i:2, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iter: 0 i:12, pairs changed 2
L==H
j not moving enough
j not moving enough
iter: 0 i:18, pairs changed 3
j not moving enough
j not moving enough
iter: 0 i:26, pairs changed 4
iter: 0 i:29, pairs changed 5
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j n

In [53]:
b

matrix([[-3.8434788]])

In [54]:
alphas[alphas>0]

matrix([[ 0.13412133,  0.2259251 ,  0.01015528,  0.37020171]])

In [55]:
shape(alphas[alphas>0])

(1L, 4L)

In [58]:
for i in range(100):
    if alphas[i]>0.0:
        print dataArr[i], labelArr[i]

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0


<img src="../src/Ch06_SVM/Img/6.4.jpg">

** 그림 6.4 간략한 형태의 SMO를 실행하여 원이 그려진 지지 벡터와 분리 초평면을 보여주는 SMO 표본 데이터 집합 **

## 6.4 전체 플랫 SMO를 이용해 최적화 속도 올리기

* <span style="color:Orange"><b>간략한 형태의 SMO</b></span>는 적은 데이터 집합에서는 잘 동작하지만, 많은 데이터 집합에서는 속도가 떨어짐
* 본 장에서는, 간략한 형태의 알고리즘을 이용하여 <span style="color:Blue"><b>전체 플랫 SMO 알고리즘</b></span>으로 변경하여 사용
<br><br>
* 최적화 부분에서 알파를 변경하되 나머지는 이전에 사용된 상태를 그대로 유지함
* 단지 다른 것은 최적화에서 사용할 알파를 선택하는 방법
* 전체 플랫은 몇 가지 경험적인(heuristic) 방법을 사용하여 속도를 개선
<br><br>
* <span style="color:Blue"><b>플랫 SMO 알고리즘</b></span> <br> 
 <b>1. 첫 번째 알파를 선택하기 위해서 외부 반복문 수행</b>
 <br> 외부 반복문은 전체 데이터 집합에서 하나, 범위가 없는 알파에서 하나씩 이들 사이를 번갈아 가며 수행
 <br> (※ 범위가 없는 알파는 범위가 0 또는 한정되지 않는 알파를 말함)<br>
 <b>2. 두 번째 알파는 첫 번째 알파가 선택된 다음 내부 반복문을 사용하여 선택</b> 
 <br> 이 알파는 최적화 하는 동안 단계 크기를 가장 크게 하는 방법으로 선택됨
 <br> 오류 값에 대한 전역 캐시(global cache)를 생성하며, 단계 크기 또는 Ei-Ej를 가장 크게 하는 알파 선택
 <br> (※ <<span style="color:Orange">b>간략한 형태의 SMO</b></span> 에서는 j를 선택한 후 오류 Ej를 계산했었음) 

#### 리스팅 6.3 전체 플랫 SMO에 대한 지지 함수

* 사용될 데이터 구조와 E 값을 캐시에 저장하는 도움 함수 3개를 가짐

In [3]:
# 중요한 값들을 모두 저장해 두기 위한 데이터 구조를 생성 (객체로 되어 있음)
# 이 객체들을 데이터 구조처럼 사용
# 입력되는 값을 함수에서 처리할 수 있도록 저장하기 위해 모든 데이터를 하나의 구조가 되도록 함
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

# 첫 번째 도움 함수
# 주어진 알파에 대한 E 값을 계산하고, 계산된 E 값을 반환
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
# 두 번째 도움 함수
# 두 번째 알파 또는 내부 반복문의 알파 선택
# 이 함수의 목적은 두 번째 알파를 선택하기 위한 것으로, 각각을 최적화하는 동안 최대가 되는 단계를 찾아내는 것
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

# 오류를 계산하고 계산한 값을 캐시에 저장
def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

#### 리스팅 6.4 전체 플랫 SMO 최적화 절차
* 의사 결정 범위를 찾기 위해 최적화 절차를 간단하게 표현

In [63]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print "L==H"; return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print "eta>=0"; return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

#### 리스팅 6.5 전체 플랫 SMO 외부 반복문

In [70]:
def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print "fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print "non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print "iteration number: %d" % iter
    return oS.b,oS.alphas

In [71]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [77]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
L==H
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
j not moving enough
fullSet, iter: 0 i:7, pairs changed 2
L==H
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
L==H
fullSet, iter: 0 i:10, pairs changed 2
L==H
fullSet, iter: 0 i:11, pairs changed 2
L==H
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
L==H
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
L==H
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
j not moving enough
fullSet, iter: 0 i:22, pairs changed 3
L==H
fullSet, iter: 0 i:23, 

<img src="../src/Ch06_SVM/Img/6.5.jpg">

** 그림 6.5 데이터 집합으로 전체 SMO 알고리즘을 실행하여 얻은 지지 벡터. 이 결과는 그림 6.4와는 약간 다르다 **

## 6.5 더 복잡한 데이터를 위해 커널 사용하기

* <span style="color:Blue"><b>커널(Kernel)</b></span>
 - 선형 데이터를 위해 사용했던 방법 중에서 이 그림의 패턴을 포착하기 위한 성능 좋은 도구 사용할 방법
 - 데이터를 분류기가 이해하기 쉬운 형태로 변환하기 위해 커널이라는 것을 사용
<br><br>
* 이번 절에서는 커널과 지지 벡터 머신을 사용하는 방법을 설명
* 이후, 커널 중 <span style="color:Blue"><b>반지름 성향(radial bias) 함수</b></span> 설명
* 이 함수를 기존 분류기에 적용

### 6.5.1 고차원의 데이터를 커널을 이용해 매핑하기

* 하나의 속성 공간(feature space)에서 다른 속성 공간으로 데이터를 변환하는 예
* <b> "하나의 속성 공간에서 다른 속성 공간으로 매핑(mapping)한다" </b>
 - 일반적으로, 이러한 매핑은 저차원의 속성 공간에서 고차원 공간으로 이동시킬 때 사용}
<br><br>
* <span style="color:Blue"><b>커널(Kernel)</b></span>
 - 하나의 속성 공간에서 다른 속성 공간으로 매핑하는 것
 - 어려운 형태의 데이터를 쉬운 형태로 변환하는 포장지나 인터페이스처럼 생각해도 됨
* SVM 최적화에 대한 장점
 - 모든 연산이 내적 측면에서 다루어질 수 있다는 것
 <br> (※ 내적은 스칼라 또는 단일 수를 산출하기 위해 두 개의 벡터를 함께 곱하는 것)
 - 간소화하지 않은 커널 함수를 가지고 내적을 대신할 수 있음
* <span style="color:Blue"><b>커널 트릭(kernel trick)</b></span> 또는 <span style="color:Blue"><b>커널 서브스테이션(kernel substation)</b></span>
 - 커널을 가지고 내적을 대신하는 것

<img src="../src/Ch06_SVM/Img/6.6.jpg">

** 그림 6.6 이 데이터 점들을 2차원 평면에서 직선만으로 분리한다는 것은 어려운 일이다. 하지만, 사각형으로 된 점과 원으로 된 점을 분리하는 어떠한 패턴이 분명 존재한다. **

### 6.5.2 반지름 성향 함수

* <span style="color:Blue"><b> 반지름 성향 함수(Radial Bias Function) </b></span>
 - 커널이며, 종종 지지 벡터 머신과 함께 사용됨
 - 하나의 벡터를 가지고 벡터 간의 거리를 기반으로 하는 하나의 스칼라를 출력하는 함수
 - 가우스(Gaussian) 확률 분포 사용

<img src="../src/Ch06_SVM/Img/gaussian.jpg">
 σ는 사용자 정의(user-define) 매개변수, "거리 범위(reach)" 또는 얼마나 빨리 0으로 떨어지는지를 결정
<br>
* <span style="color:Blue"><b> 가우스 확률 분포 </b></span>
 - 데이터의 속성 공간에서 높은 속성 공간으로 매핑
 - 이 커널을 가지고 좋은 결과를 얻을 수 있음
 <br> (<span style="color:Purple"><b>***kernelTrans()*** </b></span> 用)

#### 리스팅 6.6 커널 변환 함수

In [76]:
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)

#### 리스팅 6.7 커널을 사용하기 위해 필요한 함수 innerL()과 calcEk()의 변경

In [78]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print "L==H"; return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print "eta>=0"; return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print "j not moving enough"; return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0
    
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

### 6.5.3 검사를 위한 커널 사용

#### 리스팅 6.8 커널으로 분류하기 위한 반지름 성향 검사 함수

In [79]:
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1)) #C=200 important
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] #get matrix of only support vectors
    labelSV = labelMat[svInd];
    print "there are %d Support Vectors" % shape(sVs)[0]
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print "the training error rate is: %f" % (float(errorCount)/m)
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print "the test error rate is: %f" % (float(errorCount)/m)    

In [80]:
testRbf()

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 10
fullSet, iter: 0 i:16, pairs changed 11
fullSet, iter: 0 i:17, pairs changed 12
fullSet, iter: 0 i:18, pairs changed 13
fullSet, iter: 0 i:19, pairs changed 14
fullSet, iter: 0 i:20, pairs changed 14
fullSet, iter: 0 i:21, pairs changed 15
fullSet, iter: 0 i:22, pairs changed 15
fullSet, iter: 0 i:23, pairs changed 16
fullSet, iter: 0 i:24, pairs changed 17
j not moving enough


<img src="../src/Ch06_SVM/Img/6.7.jpg">

** 그림 6.7 사용자 정의 매개변수 k1=0.1인 반지름 성향 함수. 사용자 정의 매개변수는 각 지지 벡터에 대한 영향력을 감소시키므로 더 많은 지지 벡터가 필요하다. **

<img src="../src/Ch06_SVM/Img/6.8.jpg">

** 그림 6.8 사용자 매개변수 k1=1.3인 반지름 성향 커널 함수. 여기서는 그림 6.7보다 더 적은 수의 지지 벡터를 가지고 실험헀다. 지지 벡터는 의사 결정 영역 주변에 무리지어 나타난다. **

## 6.6 예제: 필기체 인식 예제 다시 적용하기

#### 리스팅 6.9 지지 벡터 머신 필기체 인식

In [93]:
def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('digits/trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print "there are %d Support Vectors" % shape(sVs)[0]
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print "the training error rate is: %f" % (float(errorCount)/m)
    dataArr,labelArr = loadImages('digits/testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print "the test error rate is: %f" % (float(errorCount)/m) 
    
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

In [94]:
testDigits(('rbf',20))

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 6
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 6
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 6
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
j not moving enough
fullSet, iter: 0 i:15, pairs changed 9
j not moving enough
fullSet, iter: 0 i:16, pairs changed 9
j not moving enough
fullSet, iter: 0 i:17, pairs changed 9
j not moving enough
fullSet, iter: 0 i:18, pairs changed 9
fullSet, iter: 0 i:19, pairs changed 10
fullSet, iter: 0 i:20, pairs changed 11
fullSet, iter: 0 i:21, pairs changed 12
L==H
fullSet, iter: 0 i:22, pairs changed 12
L==H
fullSet, iter:

** 표 6.1 필기체 숫자에 커널 설정을 달리한 성능 비교 **

<img src="../src/Ch06_SVM/Img/Table6.1.JPG">

## 6.7 요약

* <span style="color:Blue"><b>지지 벡터 머신</b></span>은 분류기의 한 유형
* 머신이라고 불리는 이유는, 이진수로 의사 결정을 생성하는 의사결정 머신이기 때문
* 지지 벡터들은 학습하고자 하는 것에 대해 작은 일반화 오류를 가지도록 학습
* 이러한 장점 때문에 지지 벡터 머신은 인기가 있으며, 비지도 학습에서 가장 좋은 알고리즘 중 하나로 간주
* 지지 벡터 머신은 이차 방정식의 최적화 문제를 해결함으로써 초대 마진을 구함
* 그 중, 존 플랫이 SMO 알고리즘을 소개
* SVMs의 훈련이 한 번에 두 개의 알파를 최적화함으로써 빠르게 수행되도록 함
* 간소화한 알고리즘으로 전체 플랫 알고리즘을 사용할 수 있도록 함으로써 SMO 알고리즘의 속도를 빠르게 함
* 더하여, 커널 방법 또는 커널 트릭을 이용하여 저차원 공간에서 고차원 공간으로 데이터를 매핑시킴으로써 알고리즘 속도를 더욱 빠르게 만들 수 있음
* 저차원 공간에서는 비선형 데이터를 더 높은 차원에서는 선형 문제로 해결할 수 있음
* 반지름 성향 함수는 두 벡터 간의 거리를 측정하는 인기 좋은 커널
* 지지 벡터 머신은 이진 분류기이며, 두 개보다 더 많은 분류 항목을 분류하도록 추가적으로 방법 확장 가능
* SVM의 성능은 최적화 매개변수와 커널 사용에 대한 매개변수에 민감

## Reference 

* <span style="color:#1589FF"><b> Machine Learning in Action pdf(Eng Ver)</b></span>
<br> http://www2.ift.ulaval.ca/~chaib/IFT-4102-7025/public_html/Fichiers/Machine_Learning_in_Action.pdf